# PINN by MLP (Critical Temperature of a Superconductor)

In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR # for update the learning rate

## Load datasets

In [7]:
train_data = pd.read_csv("./Critical Temperature of a Superconductor/train.csv")

In [8]:
# Extract features and target variable
X = train_data.iloc[:, :-1].values  # Features (all columns except the last)
y = train_data.iloc[:, -1].values   # Target (last column, critical temperature)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

In [9]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

## Define the MLP model

In [10]:
class MLP_PINN(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, activation=nn.ReLU):
        
        super(MLP_PINN, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), activation()]
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(activation())
            layers.append(nn.Dropout(0.2))  # Dropout to prevent overfitting
        
        layers.append(nn.Linear(hidden_dim, output_dim))  # Output layer
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)
        

In [ ]:
# Initialize model
input_dim = X_train.shape[1]  # Number of features
hidden_dim = 128  # Number of neurons in hidden layers
output_dim = 1  # Single output (critical temperature)
num_layers = 4  # Number of hidden layers

model = MLP_PINN(input_dim, hidden_dim, output_dim, num_layers)

## Loss Function

In [34]:
def rmse_loss(y_true, y_pred):
    
    mse_loss = nn.functional.mse_loss(y_pred, y_true) 
    
    return torch.sqrt(mse_loss)  

In [35]:
"""
    def physics_loss(x, y_pred):
    
    return torch.mean(residual**2)
"""

'\n    def physics_loss(x, y_pred):\n    \n    return torch.mean(residual**2)\n'

In [36]:
#def data_loss(y_true, y_pred):
    
    #return nn.rmse_loss()(y_pred, y_true)

loss_fn = rmse_loss


In [37]:
"""
def combined_loss(x, y_true, y_pred):

    data_loss_value = data_loss(y_true, y_pred)
    physics_loss_value = physics_loss(x, y_pred)

    total_loss = data_loss + lambda * physics_loss
    
    return total_loss
"""

'\ndef combined_loss(x, y_true, y_pred):\n\n    data_loss_value = data_loss(y_true, y_pred)\n    physics_loss_value = physics_loss(x, y_pred)\n\n    total_loss = data_loss + lambda * physics_loss\n    \n    return total_loss\n'

## Model Training

In [38]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #initial learning rate = 0.01
scheduler = StepLR(optimizer, step_size=10, gamma=0.1) #epoch % 10 == 0, lr * 0.1

In [39]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [40]:
# Training loop
num_epochs = 1000
best_val_loss = float('inf')  
early_stop_counter = 0  
patience = 50

for epoch in range(num_epochs):
    #training
    model.train()
    for batch_X, batch_y in train_loader:
        # Fowardpass
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)
        
        # Backwardpass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # valid
    model.eval()
    with torch.no_grad():
        val_predictions = model(X_val_tensor)
        val_loss = loss_fn(val_predictions, y_val_tensor)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0  
        best_model_state = model.state_dict()  
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

Epoch 0, Training Loss: 5.0794291496276855, Validation Loss: 19.817869186401367
Epoch 10, Training Loss: 20.40797996520996, Validation Loss: 28.750885009765625
Epoch 20, Training Loss: 18.465688705444336, Validation Loss: 17.486940383911133
Epoch 30, Training Loss: 17.28324317932129, Validation Loss: 18.315826416015625
Epoch 40, Training Loss: 12.116684913635254, Validation Loss: 18.014558792114258
Epoch 50, Training Loss: 6.143571853637695, Validation Loss: 20.818016052246094
Epoch 60, Training Loss: 8.783123970031738, Validation Loss: 18.26993179321289
Epoch 70, Training Loss: 9.648964881896973, Validation Loss: 15.19027328491211
Epoch 80, Training Loss: 2.448293447494507, Validation Loss: 16.132932662963867
Epoch 90, Training Loss: 22.842145919799805, Validation Loss: 18.034488677978516
Epoch 100, Training Loss: 28.18366050720215, Validation Loss: 19.548568725585938
Epoch 110, Training Loss: 6.024419784545898, Validation Loss: 16.884401321411133
Epoch 120, Training Loss: 4.553137302

## Evaluation

In [41]:
model.load_state_dict(best_model_state)  
model.eval()

with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = loss_fn(test_predictions, y_test_tensor)
    print(f"Test Loss (RMSE): {test_loss.item()}")

Test Loss (RMSE): 16.62873077392578
